## CS-471: Machine Learning
### **Submitted By**:
#### **Name**: Ayesh Ahmad
#### **CMS**: 365966
#### **Class**: BESE-12A
---
## Lab 5
#### Train a Decision Tree classifier considering the following learning problem: the problem of deciding whether to wait for a table at a restaurant. For this problem the output, is a Boolean variable that we will call WillWait; it is true for examples where we do wait for a table.

##### Imports and data

In [14]:
import numpy as np
import pandas as pd
import random
import time
import graphviz
from decisionTree import trainTestSplit, buildDecisionTree, decisionTreePredictions, calculateAccuracy

##### Helper Functions

In [15]:
def label_encoding(txt):
    if 'Yes' in txt:
        return 1
    else:
        return 0
    
def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def checkPurity(data):
    if len(np.unique(data[:, -1])) == 1:
        return True
    else:
        return False

def classifyData(data):
    uniqueClasses, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]

def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = np.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits

def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

def calculateEntropy(data):
    _, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
    return sum(probabilities * -np.log2(probabilities))

def calculateOverallEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)

def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in range(randomSplits):
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue

def buildDecisionTree(dataFrame, currentDepth=0, minSampleSize=2, maxDepth=1000, randomAttributes=None, randomSplits=None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes is not None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population=list(range(len(COLUMN_HEADERS) - 1)), k=randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree

def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)

def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

##### Viewing Data

In [16]:
df = pd.read_csv('data.csv')
df

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,WillWait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,NaN,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No


##### Label Encoding

In [17]:
df.Alt = df.Alt.apply(label_encoding)
df.Bar = df.Bar.apply(label_encoding)
df.Fri = df.Fri.apply(label_encoding)
df.Hun = df.Hun.apply(label_encoding)
df.Rain = df.Rain.apply(label_encoding)
df.Res = df.Res.apply(label_encoding)
df.WillWait = df.WillWait.apply(label_encoding)
df

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,WillWait
0,1,0,0,1,Some,$$$,0,1,French,0-10,1
1,1,0,0,1,Full,$,0,0,Thai,30-60,0
2,0,1,0,0,Some,$,0,0,Burger,0-10,1
3,1,0,1,1,Full,$,1,0,Thai,10-30,1
4,1,0,1,0,Full,$$$,0,1,French,>60,0
5,0,1,0,1,Some,$$,1,1,Italian,0-10,1
6,0,1,0,0,NaN,$,1,0,Burger,0-10,0
7,0,0,0,1,Some,$$,1,1,Thai,0-10,1
8,0,1,1,0,Full,$,1,0,Burger,>60,0
9,1,1,1,1,Full,$$$,0,1,Italian,10-30,0


##### One-Hot Encoding

In [18]:
df = pd.get_dummies(df)
df

,Alt,Bar,Fri,Hun,Rain,Res,WillWait,Pat_Full,Pat_Some,Price_$,Price_$$,Price_$$$,Type_Burger,Type_French,Type_Italian,Type_Thai,Est_0-10,Est_10-30,Est_30-60,Est_>60
0,1,0,0,1,0,1,1,False,True,False,False,True,False,True,False,False,True,False,False,False
1,1,0,0,1,0,0,0,True,False,True,False,False,False,False,False,True,False,False,True,False
2,0,1,0,0,0,0,1,False,True,True,False,False,True,False,False,False,True,False,False,False
3,1,0,1,1,1,0,1,True,False,True,False,False,False,False,False,True,False,True,False,False
4,1,0,1,0,0,1,0,True,False,False,False,True,False,True,False,False,False,False,False,True
5,0,1,0,1,1,1,1,False,True,False,True,False,False,False,True,False,True,False,False,False
6,0,1,0,0,1,0,0,False,False,True,False,False,True,False,False,False,True,False,False,False
7,0,0,0,1,1,1,1,False,True,False,True,False,False,False,False,True,True,False,False,False
8,0,1,1,0,1,0,0,True,False,True,False,False,True,False,False,False,False,False,False,True
9,1,1,1,1,0,1,0,True,False,False,False,True,False,False,True,False,False,True,False,False


##### Reformatting data

In [19]:
df = df[df.columns.tolist()[0: 6] + df.columns.tolist()[7: ] + df.columns.tolist()[6: 7]]
df

,Alt,Bar,Fri,Hun,Rain,Res,Pat_Full,Pat_Some,Price_$,Price_$$,Price_$$$,Type_Burger,Type_French,Type_Italian,Type_Thai,Est_0-10,Est_10-30,Est_30-60,Est_>60,WillWait
0,1,0,0,1,0,1,False,True,False,False,True,False,True,False,False,True,False,False,False,1
1,1,0,0,1,0,0,True,False,True,False,False,False,False,False,True,False,False,True,False,0
2,0,1,0,0,0,0,False,True,True,False,False,True,False,False,False,True,False,False,False,1
3,1,0,1,1,1,0,True,False,True,False,False,False,False,False,True,False,True,False,False,1
4,1,0,1,0,0,1,True,False,False,False,True,False,True,False,False,False,False,False,True,0
5,0,1,0,1,1,1,False,True,False,True,False,False,False,True,False,True,False,False,False,1
6,0,1,0,0,1,0,False,False,True,False,False,True,False,False,False,True,False,False,False,0
7,0,0,0,1,1,1,False,True,False,True,False,False,False,False,True,True,False,False,False,1
8,0,1,1,0,1,0,True,False,True,False,False,True,False,False,False,False,False,False,True,0
9,1,1,1,1,0,1,True,False,False,False,True,False,False,True,False,False,True,False,False,0


##### Creating Decision Tree

In [20]:
dataFrameTrain, dataFrameTest = trainTestSplit(df, testSize = 0.1667)

i = 1
accuracyTrain = 0
while accuracyTrain < 100:
    startTime = time.time()
    decisionTree = buildDecisionTree(dataFrameTrain, maxDepth = i)
    buildingTime = time.time() - startTime
    decisionTreeTestResults = decisionTreePredictions(dataFrameTest, decisionTree)
    accuracyTest = calculateAccuracy(decisionTreeTestResults, dataFrameTest.iloc[:, -1]) * 100
    decisionTreeTrainResults = decisionTreePredictions(dataFrameTrain, decisionTree)
    accuracyTrain = calculateAccuracy(decisionTreeTrainResults, dataFrameTrain.iloc[:, -1]) * 100
    print("maxDepth = {}: ".format(i), end = "")
    print("accTest = {0:.2f}%, ".format(accuracyTest), end = "")
    print("accTrain = {0:.2f}%, ".format(accuracyTrain), end = "")
    print("buildTime = {0:.2f}s".format(buildingTime), end = "\n")
    i += 1

maxDepth = 1: accTest = 50.00%, accTrain = 90.00%, buildTime = 0.00s
maxDepth = 2: accTest = 50.00%, accTrain = 90.00%, buildTime = 0.00s
maxDepth = 3: accTest = 50.00%, accTrain = 100.00%, buildTime = 0.01s


##### Visualizing the Decision Tree

In [27]:
def visualize_decision_tree(tree, dot_data=None):
    if dot_data is None:
        dot_data = 'digraph Tree { node [shape=box] ;'
    if isinstance(tree, dict):
        for question, subtree in tree.items():
            if isinstance(subtree, dict):
                dot_data += ' "{}" -> "{}" [labeldistance=2.5, labelangle=45, headlabel="{}"];'.format(question, list(subtree.keys())[0], list(subtree.values())[0])
                dot_data = visualize_decision_tree(list(subtree.values())[0], dot_data)
            else:
                dot_data += ' "{}" -> "{}";'.format(question, subtree)
    return dot_data + '}'

custom_decision_tree = buildDecisionTree(dataFrameTrain, maxDepth=3)
dot_data = visualize_decision_tree(custom_decision_tree)
graph = graphviz.Source(dot_data)
graph.render('decision_tree', view=True)

'decision_tree.pdf'